In [1]:
import joblib
import pandas as pd
import numpy as np
import os

# --- KONFIGURASI LOKASI MODEL ---
# Coba beberapa kemungkinan lokasi file model secara otomatis
possible_paths = [
    '../models/current_stress_pipeline.joblib'
]

model_path = None
for path in possible_paths:
    if os.path.exists(path):
        model_path = path
        break

if model_path is None:
    print("❌ ERROR FATAL: File model 'current_stress_model.joblib' tidak ditemukan!")
    print("Pastikan Anda sudah menjalankan notebook training dan file .joblib sudah terbentuk.")
    print(f"Sistem mencari di lokasi ini tetapi gagal: {possible_paths}")
    # Stop eksekusi manual agar tidak lanjut ke bawah
    raise FileNotFoundError("File model tidak ditemukan. Cek lokasi file Anda.")

print(f"✅ Model ditemukan di: {model_path}")

# 1. Load Model
try:
    artifacts = joblib.load(model_path)
    pipeline = artifacts['pipeline']
    feature_names = artifacts['feature_names']
    print("✅ Artifacts berhasil dimuat (pipeline).")
except Exception as e:
    print(f"❌ Terjadi error saat load joblib: {e}")
    raise e

# 2. Siapkan Data Input
input_data = {
    'Study_Hours_Per_Day': [6],
    'Extracurricular_Hours_Per_Day': [2],
    'Sleep_Hours_Per_Day': [4],
    'Social_Hours_Per_Day': [2],
    'Physical_Activity_Hours_Per_Day': [1],
    'GPA': [3.5],
}

df_input = pd.DataFrame(input_data)

# 3. Feature Engineering (Membuat kolom Academic_Performance dari GPA)
def categorize_academic_performance(gpa):
    if gpa >= 3.5:
        return 'Excellent'
    elif 3.0 <= gpa < 3.5:
        return 'Good'
    elif 2.0 <= gpa < 3.0:
        return 'Fair'
    else:
        return 'Poor'

# Terapkan fungsi
df_input['Academic_Performance'] = df_input['GPA'].apply(categorize_academic_performance)
print(f"ℹ️ Kategori Akademik terdeteksi: {df_input['Academic_Performance'][0]}")

# 4. Encoding (Mengubah teks menjadi angka)
mapping_performance = {'Poor': 0, 'Fair': 1, 'Good': 2, 'Excellent': 3}
df_input['Academic_Performance_Encoded'] = df_input['Academic_Performance'].map(mapping_performance)

# 5. Pilih dan Urutkan Kolom sesuai Model
# Karena kita sudah memastikan load berhasil di atas, variabel feature_names pasti ada
try:
    df_ready = df_input[feature_names]
except KeyError as e:
    print(f"❌ Error: Data input tidak cocok dengan model.")
    print(f"Model meminta: {feature_names}")
    print(f"Data yang ada: {df_input.columns.tolist()}")
    raise e

# 6. Scaling & Prediksi
try:
    prediction = pipeline.predict(df_ready)
    probability = pipeline.predict_proba(df_ready)
    
    # Mapping hasil prediksi
    stress_mapping = {0: 'Low', 1: 'Moderate', 2: 'High'}
    predicted_label = stress_mapping.get(prediction[0], prediction[0])

    print("\n" + "="*30)
    print(f"🎯 Hasil Prediksi Stress: {predicted_label}")
    print(f"📊 Probabilitas: {probability[0]}")
    print("="*30)

except Exception as e:
    print(f"❌ Terjadi error saat prediksi: {e}")

✅ Model ditemukan di: ../models/current_stress_pipeline.joblib
✅ Artifacts berhasil dimuat (pipeline).
ℹ️ Kategori Akademik terdeteksi: Excellent

🎯 Hasil Prediksi Stress: High
📊 Probabilitas: [0. 0. 1.]
